In [ ]:
!pip3 install --upgrade gtfs-realtime-bindings
!pip3 install mplleaflet

In [ ]:
from google.transit import gtfs_realtime_pb2
import urllib.request
import mplleaflet as mp
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
route = pd.read_csv('routes.txt',sep=",")
route

,route_short_name,route_long_name,route_type,route_id,agency_id
0,NaN,108DOWN,3,0,DIMTS
1,NaN,108STLDOWN,3,1,DIMTS
2,NaN,108STLUP,3,2,DIMTS
3,NaN,108UP,3,3,DIMTS
4,NaN,185DOWN,3,4,DIMTS
...,...,...,...,...,...
1540,NaN,624ACLDOWN_DTC,3,1540,DTC
1541,NaN,408CLDOWN_DTC,3,1541,DTC
1542,NaN,114UP_DTC,3,1543,DTC
1543,NaN,182ACLSTLU_DTC,3,1545,DTC


In [ ]:
trip = pd.read_csv('trips.txt',sep=',')
trip

,route_id,service_id,trip_id,shape_id
0,0,1,0_6_0,0.0
1,0,1,0_6_10,0.0
2,0,1,0_6_20,0.0
3,0,1,0_6_30,0.0
4,0,1,0_6_40,0.0
...,...,...,...,...
75752,1544,1,1544_15_25,NaN
75753,1544,1,1544_8_31,NaN
75754,1544,1,1544_17_7,NaN
75755,1544,1,1544_13_55,NaN


In [ ]:
df=route.merge(trip,how='inner',on='route_id')[['route_long_name','route_id','shape_id']]
df

,route_long_name,route_id,shape_id
0,108DOWN,0,0.0
1,108DOWN,0,0.0
2,108DOWN,0,0.0
3,108DOWN,0,0.0
4,108DOWN,0,0.0
...,...,...,...
75752,114DOWN_DTC,1544,NaN
75753,114DOWN_DTC,1544,NaN
75754,114DOWN_DTC,1544,NaN
75755,114DOWN_DTC,1544,NaN


In [ ]:
route['agency_id'].value_counts()

DIMTS    1210
DTC       335
Name: agency_id, dtype: int64

In [ ]:
shapes = pd.read_csv('shapes.txt', sep=",")
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,0,28.625198,77.110808,0
1,0,28.625220,77.110815,1
2,0,28.625280,77.110831,2
3,0,28.625483,77.110835,3
4,0,28.625705,77.110839,4
...,...,...,...,...
242239,533,28.618988,77.031115,708
242240,533,28.619000,77.031478,709
242241,533,28.619005,77.031547,710
242242,533,28.618953,77.031561,711


In [ ]:
route_shape_map = df.merge(shapes,how='inner',on='shape_id')
route_shape_map

,route_long_name,route_id,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,108DOWN,0,0.0,28.625198,77.110808,0
1,108DOWN,0,0.0,28.625220,77.110815,1
2,108DOWN,0,0.0,28.625280,77.110831,2
3,108DOWN,0,0.0,28.625483,77.110835,3
4,108DOWN,0,0.0,28.625705,77.110839,4
...,...,...,...,...,...,...
16495216,751UP,533,533.0,28.618988,77.031115,708
16495217,751UP,533,533.0,28.619000,77.031478,709
16495218,751UP,533,533.0,28.619005,77.031547,710
16495219,751UP,533,533.0,28.618953,77.031561,711


# Graph creation

In [ ]:
x = route_shape_map.groupby(['shape_pt_lat','shape_pt_lon']).size()
x

shape_pt_lat  shape_pt_lon
28.490695     77.304077       100
28.490785     77.303741       880
28.490846     77.304046       409
28.491244     77.303870       966
28.491317     77.304038        93
                             ... 
28.858176     77.093328        35
              77.093332        35
              77.093337        35
              77.093342        35
              77.093347        35
Length: 46033, dtype: int64

In [ ]:
graph = {}

In [ ]:
for index,row in route_shape_map.iterrows():
  gps = (row['shape_pt_lat'],row['shape_pt_lon'])
  if gps in graph:
    if row['route_id'] not in graph[gps][route]:
      graph[gps]['route'].append(row['route_id'])
    
  else:
    graph[gps]['route']=[]
    graph[gps]['route'].append(row['route_id'])



route_long_name      108DOWN
route_id                   0
shape_id                   0
shape_pt_lat         28.6252
shape_pt_lon         77.1108
shape_pt_sequence          0
Name: 0, dtype: object


#Check


In [ ]:
def put(feed,lat,lon):
  # print(feed.entity)
  for entity in feed.entity:
    if entity.id=="DL1PC6641":
      lat.append(entity.vehicle.position.latitude)
      lon.append(entity.vehicle.position.longitude)
      break

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
feed.ParseFromString(response.read())
print(feed.entity)

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
lat=[]
lon=[]
for i in range(100):
  response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
  feed.ParseFromString(response.read())
  put(feed,lat,lon)
  time.sleep(10)

In [ ]:
lat

In [ ]:
fig =plt.figure()
plt.plot(lon,lat,linewidth=3)
mp.display(fig=fig)